In [85]:
#time delay要考慮系統塞車問題
#搜尋改成搜尋關鍵字給使用者輸入

import requests
from bs4 import BeautifulSoup as bs
from seleniumwire import webdriver
from selenium.webdriver.common.by import By
import time
import re
import json

account = 'cindy1999881114@gmail.com'
password = 'chenyingshan19991114'
activity_page = 'https://tixcraft.com/activity/detail/23_tvl'

def login(acc, pwd):
    driver = webdriver.Chrome()
    driver.get(activity_page)

    #driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]').click()
    #driver.implicitly_wait(1)
    driver.find_element(By.XPATH, '//*[@id="bs-navbar"]/div/div[2]/ul[3]/li/a/span').click()
    time.sleep(0.2)
    driver.find_element(By.XPATH, '//*[@id="facebook"]').click()

    #time.sleep(1)
    acc_driver = driver.find_element(By.XPATH, '//*[@id="email"]')
    acc_driver.clear()
    acc_driver.send_keys(acc)
    pwd_driver = driver.find_element(By.XPATH, '//*[@id="pass"]')
    pwd_driver.clear()
    pwd_driver.send_keys(pwd)
    driver.find_element(By.XPATH, '//*[@id="loginbutton"]').click()

    return driver

driver = login(account, password)
driver.refresh()

for request in driver.requests:
  if (request.url == activity_page and request.headers['sec-fetch-site'] == 'cross-site'):
    activity_link = request.url  # Request url
    activity_header = dict(request.headers)  # Request headers
    print('login!!')
#driver.quit()

session_requests = requests.session()
response = session_requests.request("GET", url = activity_link, headers = activity_header)
soup = bs(response.text, 'html.parser')


login!!


In [79]:

print('你選擇的是搶這個活動的票:' + activity_link)
# Activity page
#response = session_requests.request("GET", url = request.url, headers = request.headers)
#soup = bs(response.text, 'html.parser')
link = soup.find_all('li', class_='buy')
#print(soup)
buy_now = 'https://tixcraft.com' + link[0].a.get("href")
print('進這個網站選場次:' + buy_now)

# Choose events page
find_tickets = ''
response = session_requests.request("GET", url = buy_now)
soup = bs(response.text, 'html.parser')
link_list = soup.find_all('tbody')
links = link_list[0].find_all('tr')
links_length = len(links)
links_index = 0
while (find_tickets == '' and links_index < links_length): #目前這個條件式，會選擇第一個仍有票的button
    tds = links[links_index].find_all('td')
    event_date = tds[0].text
    button = tds[3].find_all('button')
    #print(links[links_index].get('data-key'))
    if button != []:
        find_tickets = button[0].get('data-href')
    else:
        #print(tds[3].text)
        links_index += 1
if (links_index == links_length):
    print('每個場次都賣光了')
print('進這個網站選位置:' + find_tickets)

# Choose seats page
response = session_requests.request("GET", url = find_tickets)
soup = bs(response.text, 'html.parser')

# Use json to load area link
script = soup.find_all('script', text=re.compile('areaUrlList'))[0].text
data = re.search('areaUrlList = (.*);', script).group(1)
json_data = json.loads(data)

zone_area_lists = soup.find_all('div', class_ = 'zone area-list')
for zone_area_list in zone_area_lists:
    area_lists = zone_area_list.find_all('ul', class_ = 'area-list')
    for area_list in area_lists:
        seats = area_list.find_all('a')
        for seat in seats:
            id = seat.get('id')
            seat_name = seat.text
            seat_link = json_data[id]

seat_link = json_data[area_lists[0].find_all('a')[0].get('id')]
print('進這個網站選張數:' + seat_link)
# Select tickets
            


你選擇的是搶這個活動的票:https://tixcraft.com/activity/detail/23_tvl
進這個網站選場次:https://tixcraft.com/activity/game/23_tvl
進這個網站選位置:https://tixcraft.com/ticket/area/23_tvl/15291
進這個網站選張數:https://tixcraft.com/ticket/ticket/23_tvl/15291/1/25


In [80]:
response = session_requests.request("GET", url = seat_link)
soup = bs(response.text, 'html.parser')

payload = {}
# get csrf
csrf_name = soup.find('head').find('meta', attrs={'name': 'csrf-param'})['content']
csrf_token = soup.find('head').find('meta', attrs={'name': 'csrf-token'})['content']
payload[csrf_name] = csrf_token

# get tickets
ticket_price_list = soup.find('table', id = 'ticketPriceList').find('tbody').find_all('tr')
for ticket_price in ticket_price_list:
    payload[ticket_price.find('select')['name']] = 1
    payload[ticket_price.find('input')['name']] = ticket_price.find('input')['value']

# get verification code
verify_url = soup.find('div', class_='row mgt-32 mb-2').find('img')['src']
verify_name = soup.find('div', class_='row mgt-32 mb-2').find('input')['name']
verify_code = '0000'
payload[verify_name] = verify_code

# get agree
agree_name = soup.find('div', class_='form-check col-lg-12 col-md-12 col-sm-12 col-xs-12 col-12').find('input')['name']
agree_value = soup.find('div', class_='form-check col-lg-12 col-md-12 col-sm-12 col-xs-12 col-12').find('input')['value']
payload[agree_name] = agree_value
                          
print('https://tixcraft.com' + verify_url)

# payload
print(payload)

https://tixcraft.com/ticket/captcha?v=65c330ee94df55.88676676
{'_csrf': 'DHHkHjxq6Zpft-XD6Nf4FzioJIyj_uoz3z58Ofic9Hw7JJdWST2rymvBiJfbjbl1fv5HxNOhr0GYT0VRzN2xHg==', 'TicketForm[ticketPrice][02]': 1, 'TicketForm[priceSize][02]': '1', 'TicketForm[ticketPrice][03]': 1, 'TicketForm[priceSize][03]': '1', 'TicketForm[ticketPrice][04]': 1, 'TicketForm[priceSize][04]': '1', 'TicketForm[ticketPrice][05]': 1, 'TicketForm[priceSize][05]': '1', 'TicketForm[verifyCode]': '0000', 'TicketForm[agree]': '1'}


In [81]:
from PIL import Image
import pytesseract

url = 'https://tixcraft.com' + verify_url
response = session_requests.get(url).content
with open('test.png','wb') as f:
    f.write(response)
f.close()
img = Image.open('test.png')

pytesseract.pytesseract.tesseract_cmd = r'/Users/yingshanchen/miniconda3/envs/crawler/bin/tesseract'
text = pytesseract.image_to_string(img, lang='eng')
img.show()
print(text)

In [22]:
from ddddocr import DdddOcr
with open('test.png', 'rb') as f:
    ima_bytes = f.read()

res = DdddOcr().classification(ima_bytes)
print(res)

欢迎使用ddddocr，本项目专注带动行业内卷，个人博客:wenanzhe.com
训练数据支持来源于:http://146.56.204.113:19199/preview
爬虫框架feapder可快速一键接入，快速开启爬虫之旅：https://github.com/Boris-code/feapder
谷歌reCaptcha验证码 / hCaptcha验证码 / funCaptcha验证码商业级识别接口：https://yescaptcha.com/i/NSwk7i
zeto


'\npayload = {\n    \'_csrf\': \'_-b5d9XcDKVOAFvZBcKdIjxnKi68wwq7NPW0nPfj9pyRhMsUmoNh63d5MbRhg95DfhNce8-HbY1Zr-TwoYyp2g==\',\n    \'TicketForm[ticketPrice][02]\': \'1\',\n    \'TicketForm[priceSize][02]\': \'1\',\n    \'TicketForm[ticketPrice][03]\': \'0\',\n    \'TicketForm[priceSize][03]\': \'1\',\n    \'TicketForm[ticketPrice][04]\': \'1\',\n    \'TicketForm[priceSize][04]\': \'1\',\n    \'TicketForm[ticketPrice][05]\': \'2\',\n    \'TicketForm[priceSize][05]\': \'1\',\n    \'TicketForm[verifyCode]\': \'liwa\',\n    \'TicketForm[agree]\': \'1\'\n}\nresponse = requests.request("POST", url = seat_link, params= payload)\n'

In [82]:
print(activity_header)
payload[verify_name] = 'diyu'#text

'''
payload format
payload = {
    '_csrf': '_-b5d9XcDKVOAFvZBcKdIjxnKi68wwq7NPW0nPfj9pyRhMsUmoNh63d5MbRhg95DfhNce8-HbY1Zr-TwoYyp2g==',
    'TicketForm[ticketPrice][02]': '1',
    'TicketForm[priceSize][02]': '1',
    'TicketForm[ticketPrice][03]': '0',
    'TicketForm[priceSize][03]': '1',
    'TicketForm[ticketPrice][04]': '1',
    'TicketForm[priceSize][04]': '1',
    'TicketForm[ticketPrice][05]': '2',
    'TicketForm[priceSize][05]': '1',
    'TicketForm[verifyCode]': 'liwa',
    'TicketForm[agree]': '1'
}
'''

# payload
print(payload)
activity_header['sec-fetch-site'] = 'same-origin'
activity_header['referer'] = seat_link
activity_header['origin'] = 'http://tixcraft.com'
activity_header['content-length'] = '450'
activity_header['content-type'] = 'application/x-www-form-urlencode'
print(activity_header)
response = session_requests.request("POST", url = seat_link, headers = activity_header, data = payload)
#print(response.text)
if response.status_code == 200:
    print('move')
if response.status_code == 301:
    print('moved')
response = session_requests.request("GET", url = 'https://tixcraft.com/ticket/order', headers = activity_header, allow_redirects=False)
if response.status_code == 200:
    print('ordering')

activity_header['referer'] = 'https://tixcraft.com/ticket/order'
response = session_requests.request("GET", url = 'https://tixcraft.com/ticket/checkout', headers =  activity_header)
if response.status_code == 200:
    print('check')

{'cache-control': 'max-age=0', 'sec-ch-ua': '"Not A(Brand";v="99", "Google Chrome";v="121", "Chromium";v="121"', 'sec-ch-ua-mobile': '?0', 'sec-ch-ua-platform': '"macOS"', 'upgrade-insecure-requests': '1', 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36', 'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7', 'sec-fetch-site': 'cross-site', 'sec-fetch-mode': 'navigate', 'sec-fetch-user': '?1', 'sec-fetch-dest': 'document', 'referer': 'https://www.facebook.com/', 'accept-encoding': 'gzip, deflate, br', 'accept-language': 'en-US,en;q=0.9', 'cookie': 'tagHash=; _ga=GA1.1.1462290921.1707290827; SID=lrlid4dp9bjteqhlm0hideke4h; _csrf=e8a41d5bd28582c9094d5f93eee741de7d70b4d9ff08eb43c512351a09238492a%3A2%3A%7Bi%3A0%3Bs%3A5%3A%22_csrf%22%3Bi%3A1%3Bs%3A32%3A%22TgtRqp463U8SkyOgEKh1HXsbEG0nQjbd%22%3B%7D; _ga_C3KRPGTSF6=GS